In [1]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Sat Dec 28 19:47:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              44W / 350W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
#@title # Setup
%cd /content/
!apt install lshw
!curl -fsSL https://ollama.com/install.sh | sh
!git clone --depth 1 https://github.com/chigkim/Ollama-MMLU-Pro
%cd Ollama-MMLU-Pro
!pip install -r requirements.txt


/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
fatal: destination path 'Ollama-MMLU-Pro' already exists and is not an empty directory.
/content/Ollama-MMLU-Pro


In [3]:
#@title # Run server

%env OLLAMA_NUM_PARALLEL=16
%env OLLAMA_FLASH_ATTENTION=1

import threading
import subprocess
server = threading.Thread(target=lambda: subprocess.run(['ollama', 'serve'], check=True), daemon=True)
server.start()
import  time

time.sleep(2)

env: OLLAMA_NUM_PARALLEL=16
env: OLLAMA_FLASH_ATTENTION=1


In [4]:
#@title Download
print("Downloading model...")
!ollama pull llama3:8b-instruct-q8_0 &>/dev/null


In [5]:
#@title Run Test

!rm -rf eval_results
!python run_openai.py \
    --url http://localhost:11434/v1 \
    --model llama3:8b-instruct-q8_0 \
    --category 'computer science' \
    --verbosity 0 \
    --parallel 16


2024-12-28 19:49:17.470341
{
	"comment": "",
	"server": {
		"url": "http://localhost:11434/v1",
		"model": "llama3:8b-instruct-q8_0",
		"timeout": 600.0
	},
	"inference": {
		"temperature": 0.0,
		"top_p": 1.0,
		"max_tokens": 2048,
		"system_prompt": "The following are multiple choice questions (with answers) about {subject}. Think step by step and then finish your answer with \"the answer is (X)\" where X is the correct letter choice.",
		"style": "multi_chat"
	},
	"test": {
		"subset": 1.0,
		"parallel": 16
	},
	"log": {
		"verbosity": 0,
		"log_prompt": true
	}
}
assigned subjects ['computer science']
Testing computer science...
100% 410/410 [08:20<00:00,  1.22s/it]
Finished testing computer science in 8 minutes 20 seconds.
Total, 163/410, 39.76%
Random Guess Attempts, 8/410, 1.95%
Correct Random Guesses, 2/8, 25.00%
Adjusted Score Without Random Guesses, 161/402, 40.05%
Finished the benchmark in 8 minutes 22 seconds.
Total, 163/410, 39.76%
Random Guess Attempts, 8/410, 1.95%
Corre